### [项目在线代码](https://github.com/ksm26/AI-Agentic-Design-Patterns-with-AutoGen/blob/main/L1_Multi-Agent_Conversation_and_Stand-up_Comedy.ipynb)
### [视频地址](https://www.bilibili.com/video/BV1WJ4m137dk/?spm_id_from=333.788.videopod.episodes&vd_source=1d3a7b81d826789081d8b6870d4fff8e&p=2)
### [文档](https://microsoft.github.io/autogen/0.2/docs/Getting-Started)

![attachment:image.png](https://github.com/ksm26/AI-Agentic-Design-Patterns-with-AutoGen/blob/main/images/l1.png?raw=true)

In [1]:
import sys
sys.path.append('/opt/project/KG_Assist_LLM')

### 调用ollama

In [2]:
config_list = [
  {
    "model": "qwen2.5:72b-instruct-q4_0",
    "base_url": "http://192.168.28.5:11434/v1",
    "api_key": "ollama",
    "price": [0, 0], 
  }
]


### 调用openai本地

http://192.168.28.5:8178/v1/

In [3]:
config_list = [
    {
        "model": "qwen",
        "api_key": "sk-xxx",
        "api_type": "openai",
        "base_url": "http://192.168.28.5:8178/v1",
        "n": 1,
        "max_tokens": 4096,
        "temperature": 0,
        "top_p": 0.9,
        "price": [0, 0], 
        "default_headers": {
            "Content-types": "application/json",
            "Authorization": "Bearer xxxx"
        }
    }
]

调用 openai-hk

In [4]:
config_list = [
    {
        "model": "gpt-3.5-turbo",
        "api_key": "hk-xxx",
        "api_type": "openai",
        "base_url": "https://api.openai-hk.com/v1",
        "n": 1,
        "max_tokens": 4096,
        "temperature": 0,
        "top_p": 0.9,
        "price": [0, 0], 
        "default_headers": {
            "Content-types": "application/json",
            "Authorization": "Bearer hk-xxx"
        }
    }
]

gpt4free

- models: https://github.com/xtekky/gpt4free/blob/main/docs/providers-and-models.md#models

In [5]:
# config_list = [
#     {
#         "model": "gpt-3.5-turbo",
#         "api_key": "",
#         "api_type": "openai",
#         "base_url": "http://192.168.28.5:1337/v1",
#         "default_headers": {
#                     "Content-types": "application/json",
#         }
#     }
# ]

In [6]:
llm_config={"config_list": config_list}

In [7]:
from autogen import ConversableAgent

agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER", # always 
)

我们让 ai 回答一个笑话

In [8]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)

Sure, here's a joke for you:

Why couldn't the bicycle stand up by itself?

Because it was two tired!


然后让 ai 重复这个笑话，单个聊天场景下，ai 将不清楚之前问过的笑话，会询问这个笑话是啥，而不是重复之前的笑话

In [9]:
reply = agent.generate_reply(
    messages=[{"content": "Repeat the joke please.", "role": "user"}]
)
print(reply)

Of course! Here's the joke again: 

Why did the scarecrow win an award? Because he was outstanding in his field!


### Conversation
- Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.
- 我们创建两个对话 agent，这两个 agent能够相互对话并且具有记忆能力。

In [10]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.", # 接着上一个笑话的笑点往下说笑话
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [11]:
chat_result = joe.initiate_chat(
    recipient=cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Thanks for setting me up for some laughs. So, I recently started a new diet where I only eat food that starts with the letter "P." Let me tell you, it's been a real struggle trying to find a way to fit pizza, pasta, and pancakes into every meal!

--------------------------------------------------------------------------------
joe (to cathy):

That's hilarious, Cathy! It sounds like you're on the "P"izza, "P"asta, and "P"ancakes diet!

--------------------------------------------------------------------------------
cathy (to joe):

Cathy

--------------------------------------------------------------------------------


### Print some results
You can print out:

1. Chat history
2. Cost
3. Summary of the conversation

输出聊天历史记录

In [12]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling.",
  'name': 'joe',
  'role': 'assistant'},
 {'content': 'Hey Joe! Thanks for setting me up for some laughs. So, I '
             'recently started a new diet where I only eat food that starts '
             'with the letter "P." Let me tell you, it\'s been a real struggle '
             'trying to find a way to fit pizza, pasta, and pancakes into '
             'every meal!',
  'name': 'cathy',
  'role': 'user'},
 {'content': 'That\'s hilarious, Cathy! It sounds like you\'re on the "P"izza, '
             '"P"asta, and "P"ancakes diet!',
  'name': 'joe',
  'role': 'assistant'},
 {'content': 'Cathy', 'name': 'cathy', 'role': 'user'}]


资源消耗统计

In [13]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'total_cost': 0},
 'usage_including_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 94,
                                                             'cost': 0.0,
                                                             'prompt_tokens': 298,
                                                             'total_tokens': 392},
                                      'total_cost': 0.0}}


对话摘要生成（通常可能是最后一句话）

In [14]:
pprint.pprint(chat_result.summary)

'Cathy'


### Get a better summary of the conversation

summary_method和summary_prompt参数会在最后调用，作为 promot 对前面的对话内容进行更的总结

In [15]:
chat_result = joe.initiate_chat(
    cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.", 
    max_turns=2, 
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Thanks for setting me up for some laughs. So, I recently started a new diet where I only eat food that starts with the letter "P." Let me tell you, it's been a real struggle trying to find a way to fit pizza, pasta, and pancakes into every meal!

--------------------------------------------------------------------------------
joe (to cathy):

That's hilarious, Cathy! It sounds like you're on the "P"izza, "P"asta, and "P"ancakes diet!

--------------------------------------------------------------------------------
cathy (to joe):

Cathy

--------------------------------------------------------------------------------


In [16]:
pprint.pprint(chat_result.summary)

('Cathy is joking about being on a diet where she only eats foods that start '
 'with the letter "P," like pizza, pasta, and pancakes.')


### Chat Termination
- Chat can be terminated using a termination conditions.
- 如果我们不知道多少轮结束后会结束， 可以设置一些人为条件来提前控制

In [17]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    # 这里给大模型一个提示， 就是当你说I gotta go的时候，就直接结束会话。
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    # 这里是结束对话的条件函数，当会话中包含I gotta go，就会结束会话。
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [18]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe! Ready to laugh? Why did the scarecrow win an award? Because he was outstanding in his field!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a good one, Cathy! Here's one for you: Why did the math book look sad? Because it had too many problems.

--------------------------------------------------------------------------------
cathy (to joe):

Haha, I love it, Joe! Here's another one for you: Why did the tomato turn red? Because it saw the salad dressing!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a classic, Cathy! Here's one more for you: Why couldn't the bicycle stand up by itself? Because it was two tired!

-------------------------------------------------------------------------------

In [19]:
cathy.send(message="What's last joke we talked about?", recipient=joe)

cathy (to joe):

What's last joke we talked about?

--------------------------------------------------------------------------------
joe (to cathy):

The last joke we talked about was "Why couldn't the bicycle stand up by itself? Because it was two tired!"

--------------------------------------------------------------------------------
cathy (to joe):

Haha, that's right! Thanks for reminding me, Joe. I gotta go.

--------------------------------------------------------------------------------
